# First thing to do, make a copy of this notebook under File->Make a Copy with some intelligent naming convention.

---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [1]:
from datetime import datetime
import numpy as np
tstart = '2017-10-23T00:00:00'
tend = '2017-10-24T00:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')                

---
# Step 1, get the nominal RA/Dec Position for the Moon at the start.

## We use this to determine what the occultation times are.

In [2]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('de436.bsp')

astro_time = Time(tstart)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()


occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3}:00:00:00 {4}:{5}:00:00:00 lunar_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday, 
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/lunar_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)



../orbit_engine/lunar_296to297.sh


---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced above.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [3]:
from nustar_planning import io
occfile= "../orbit_engine/lunar_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

#print(occfile)
orbits = io.parse_occ(occfile)

# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits.head()


,visible,occulted
0,2017-10-23 00:38:39,2017-10-23 01:35:01
1,2017-10-23 02:15:19,2017-10-23 03:11:41
2,2017-10-23 03:51:58,2017-10-23 04:48:20
3,2017-10-23 05:28:38,2017-10-23 06:25:00
4,2017-10-23 07:05:17,2017-10-23 08:01:39


In [4]:
#from nustar_planning.lunar_planning import position, position_shift
import nustar_planning.lunar_planning as lunar_planning
from importlib import reload
reload(lunar_planning)

<module 'nustar_planning.lunar_planning' from '/Users/bwgref/science/local/anaconda/lib/python3.5/site-packages/nustar_planning-0.1.dev49-py3.5.egg/nustar_planning/lunar_planning.py'>

In [12]:
reload(lunar_planning)
sub = orbits.iloc[4:5].reset_index()
lunar_planning.position(sub, steps = 3)

Aim Time            RA         Dec
2017-10-23T07:05:17 248.45481  -17.11902
2017-10-23T07:24:04.333333 248.61562  -17.14604
2017-10-23T07:42:51.666667 248.77645  -17.17294


In [27]:
# PA == 0 (default), Rmoon = 940 arcseconds (default), 6 arcminutes maximum drift (default) and
# no minimum dwell period (default)
reload(lunar_planning)
lunar_planning.position_shift(sub, parallax_correction=True, pa = 202.3*u.deg, dt = 3*u.s, diag=True,
                              outfile='lunar_20171023.txt')

Start of dwell: 2017-10-23T07:00:17
End of dwell: 2017-10-23T07:02:23

Time used to aim: 2017-10-23T07:01:20
Dwell Duration (sec): 126.00 
  Dwell Drift (arcmin): 6.03

2017-10-23T07:00:17 RA: 249.44186  Dec: -16.78928

Start of dwell: 2017-10-23T07:02:23
End of dwell: 2017-10-23T07:04:50

Time used to aim: 2017-10-23T07:03:36.500000
Dwell Duration (sec): 147.00 
  Dwell Drift (arcmin): 6.09

2017-10-23T07:02:23 RA: 249.52877  Dec: -16.84752

Start of dwell: 2017-10-23T07:04:50
End of dwell: 2017-10-23T07:07:50

Time used to aim: 2017-10-23T07:06:20
Dwell Duration (sec): 180.00 
  Dwell Drift (arcmin): 6.05

2017-10-23T07:04:50 RA: 249.60213  Dec: -16.92011

Start of dwell: 2017-10-23T07:07:50
End of dwell: 2017-10-23T07:11:47

Time used to aim: 2017-10-23T07:09:48.500000
Dwell Duration (sec): 237.00 
  Dwell Drift (arcmin): 6.03

2017-10-23T07:07:50 RA: 249.64013  Dec: -17.01381

Start of dwell: 2017-10-23T07:11:47
End of dwell: 2017-10-23T07:16:08

Time used to aim: 2017-10-23T07:13: